#About the Data

* The data was taken from https://www.finra.org/rules-guidance/key-topics 

* The Data consist of three columns: Rule, Text and Topic.
  - Rule: FINRA Rule number. 
  - Text: List of rules.
  - Topic: Category to which the rule belongs.

* Each rule under the Key Topics: <br>

  1. Anti-Money Laundering
  2. Broker-Dealer Recruitment Disclosures: Complying with FINRA Rule 2273
  3. Business Continuity Planning
  4. Electronic Blue Sheets 
  5. Senior Investors
  6. Subordination Agreements 

are taken as a text.

* The rules are summarized with no duplicate text and the character count is not more than 512.
* Blank text is not considered.
* Prefixes in the beginning of the rules are removed.
* The word ‘and’ to indicate the last rule is removed.
* All sentences are considered.


# Text Preprocessing

* Duplicate ‘Text’ if any are removed.




## Next Steps

* Fine-tuning DistilBERT model for text classification





##Google Drive access

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Write the appropriate paths to retrieve the data and store results 
data_path  = '/content/drive/MyDrive/Full_Code/FINRA/Strategy2/Six_Labels/Dataset/Text_file_Strategy2_six_classes.csv'
saved_path = '/content/drive/MyDrive/Full_Code/FINRA/Strategy2/Six_Labels/'

## Set-up environment

Install the libraries folium version 0.2.1 and HuggingFace Datasets

In [ ]:
#Install the package folium version 0.2.1 and HuggingFace datasets library
!pip install -q folium==0.2.1 datasets 

     |████████████████████████████████| 69 kB 2.6 MB/s 
     |████████████████████████████████| 346 kB 8.9 MB/s 
     |████████████████████████████████| 1.1 MB 52.2 MB/s 
     |████████████████████████████████| 212 kB 57.5 MB/s 
     |████████████████████████████████| 86 kB 4.6 MB/s 
     |████████████████████████████████| 140 kB 39.8 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 52.6 MB/s 
     |████████████████████████████████| 127 kB 51.9 MB/s 
     |████████████████████████████████| 271 kB 57.7 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 144 kB 52.8 MB/s 


In [ ]:
import torch
#Check if GPU is available
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device, "is available")

cpu is available


##Memory Allocated

In [ ]:
!free -h --si | awk  '/Mem:/{print $2}'

13G


# Loading the dataset

In [ ]:
#Load the data
import pandas as pd

df = pd.read_csv(data_path)
#Display the first five rows in df
df.head()


,Rule,Text,Topic
0,3310,Each member shall develop and implement a writ...,Anti-Money Laundering
1,3310,The anti-money laundering programs required by...,Anti-Money Laundering
2,3310,Provide for annual (on a calendar-year basis) ...,Anti-Money Laundering
3,3310,The anti-money laundering programs required by...,Anti-Money Laundering
4,3310,The anti-money laundering programs required by...,Anti-Money Laundering


In [ ]:
#Display information about the dataframe df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rule    91 non-null     int64 
 1   Text    91 non-null     object
 2   Topic   91 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.3+ KB


In [ ]:
#Convert the data type of 'Text' to string
df['Text'] = df['Text'].astype(str)
#Display information about the dataframe df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rule    91 non-null     int64 
 1   Text    91 non-null     object
 2   Topic   91 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.3+ KB


In [ ]:
#Define a function to calculate the number of words in a text
def count_words(example):
  number_of_words = len(example.split()) 
  return number_of_words 


#Calculate the number of words for each 'Text' row in the dataframe df
df['number_of_words_Text'] = df['Text'].map(lambda row: count_words(row))
#Display Descriptive statistics about the 'number_of_words_Text' column in the dataframe df
df['number_of_words_Text'].describe(include='all') 

count    91.000000
mean     59.901099
std      15.566385
min      15.000000
25%      51.500000
50%      63.000000
75%      73.000000
max      84.000000
Name: number_of_words_Text, dtype: float64

In [ ]:
#Drop Duplicate entries in 'Text' if there exist any in the dataframe df
unique_df  = df.drop_duplicates(subset=['Text'])
unique_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91 entries, 0 to 90
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Rule                  91 non-null     int64 
 1   Text                  91 non-null     object
 2   Topic                 91 non-null     object
 3   number_of_words_Text  91 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 3.6+ KB


In [ ]:
#Create dataframe data with the columns 'Text' and 'Topic' from unique_df
data = unique_df[['Text','Topic']].copy()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91 entries, 0 to 90
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    91 non-null     object
 1   Topic   91 non-null     object
dtypes: object(2)
memory usage: 2.1+ KB


In [ ]:
#Reset the index in data
data = data.reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    91 non-null     object
 1   Topic   91 non-null     object
dtypes: object(2)
memory usage: 1.5+ KB


In [ ]:
#Convert the Data into HuggingFace Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(data)
dataset


Dataset({
    features: ['Text', 'Topic'],
    num_rows: 91
})

Let's look at the features of the dataset

In [ ]:
dataset.features

{'Text': Value(dtype='string', id=None),
 'Topic': Value(dtype='string', id=None)}

The dataset has to be split into training, validation and test set. Let's check the first example of the dataset:



In [ ]:
example = dataset[0]
example

{'Text': "Each member shall develop and implement a written anti-money laundering program reasonably designed to achieve and monitor the member's compliance with the requirements of the Bank Secrecy Act (31 U.S.C. 5311, et seq.), and the implementing regulations promulgated thereunder by the Department of the Treasury. Each member's anti-money laundering program must be approved, in writing, by a member of senior management.",
 'Topic': 'Anti-Money Laundering'}

Let's sort the dataset by Topic name

In [ ]:
dataset= dataset.sort('Topic')

Let's rename the column Topic to label

In [ ]:
dataset = dataset.rename_column("Topic", "label")
dataset

Dataset({
    features: ['Text', 'label'],
    num_rows: 91
})

In [ ]:
import collections 
#Define the function to check the frequency count of elements in the list
def frequency_count(mylist):
  frequency = collections.Counter(mylist)
  [print(key,':',value) for key, value in frequency.items()]

In [ ]:
print("The Frequency of label in the Dataset : \n")
frequency_count(dataset['label'])

The Frequency of label in the Dataset : 

Anti-Money Laundering : 10
Broker-Dealer Recruitment Disclosures: Complying with FINRA Rule 2273 : 6
Business Continuity Planning : 11
Electronic Blue Sheets : 25
Senior Investors : 29
Subordination Agreements : 10


The dataset consists of 6 labels.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [ ]:
labels = sorted(list(set(dataset['label'])))
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Anti-Money Laundering',
 'Broker-Dealer Recruitment Disclosures: Complying with FINRA Rule 2273',
 'Business Continuity Planning',
 'Electronic Blue Sheets',
 'Senior Investors',
 'Subordination Agreements']

## Splitting the Data into Train, Validation and Test set






In [ ]:
fix_seed = 42
from sklearn.model_selection import train_test_split

# Split data into train and val_test set 
X= dataset['Text']
y= dataset['label']
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.39,stratify=y ,random_state=fix_seed)

# Split data into val and test set 
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.50, stratify = y_val_test ,random_state=fix_seed)

In [ ]:
# Create the HuggingFace Dataset train_data
dict_train = {"sentence": X_train,"label": y_train}
train_data = Dataset.from_dict(dict_train)
train_data

Dataset({
    features: ['sentence', 'label'],
    num_rows: 55
})

In [ ]:
#Create the HuggingFace Dataset val_data
dict_val = {"sentence": X_val,"label":y_val}
val_data = Dataset.from_dict(dict_val)
val_data

Dataset({
    features: ['sentence', 'label'],
    num_rows: 18
})

In [ ]:
#Create the HuggingFace Dataset test_data
dict_test = {"sentence": X_test,"label":y_test}
test_data = Dataset.from_dict(dict_test)
test_data

Dataset({
    features: ['sentence', 'label'],
    num_rows: 18
})

Let's Check the Frequency Count of Train, Validation and Test label data

In [ ]:
#Print the frequency count of label in train_data
print("The Frequency of label in train_data : \n")
frequency_count(train_data['label'])

The Frequency of label in train_data : 

Business Continuity Planning : 7
Electronic Blue Sheets : 15
Anti-Money Laundering : 6
Subordination Agreements : 6
Senior Investors : 17
Broker-Dealer Recruitment Disclosures: Complying with FINRA Rule 2273 : 4


In [ ]:
#Print the frequency count of label in val_data
print("The Frequency of label in val_data : \n")
frequency_count(val_data['label'])

The Frequency of label in val_data : 

Subordination Agreements : 2
Anti-Money Laundering : 2
Electronic Blue Sheets : 5
Senior Investors : 6
Business Continuity Planning : 2
Broker-Dealer Recruitment Disclosures: Complying with FINRA Rule 2273 : 1


In [ ]:
#Print the frequency count of label in test_data
print("The Frequency of label in test_data : \n")
frequency_count(test_data['label'])

The Frequency of label in test_data : 

Subordination Agreements : 2
Electronic Blue Sheets : 5
Senior Investors : 6
Anti-Money Laundering : 2
Broker-Dealer Recruitment Disclosures: Complying with FINRA Rule 2273 : 1
Business Continuity Planning : 2


In [ ]:
#Create dataset_clean to store the train_data, val_data and test_data
from datasets.dataset_dict import DatasetDict
dataset_clean = DatasetDict({
    'train': train_data,
    'validation': val_data,
    'test': test_data
})
dataset_clean

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 55
    })
    validation: Dataset({
        features: ['sentence', 'label'],
        num_rows: 18
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 18
    })
})

## Save the Data

In [ ]:
#Save the HuggingFace Dataset dataset_clean in drive
dataset_clean.save_to_disk(saved_path  + "dataset_clean")
print("\nSaved dataset_clean")


Saved dataset_clean
